In [1]:
from acevedo_clss_and_fcns import * 
model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data                          = pd.read_parquet("./results/dataframes/augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./metabolites_data/metabolite_names.csv")
grafo_nx             = cobra_to_networkx(model)


/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [22]:




w  = dict(zip(grafo_nx.edges() , itertools.repeat(1)))

nx.set_edge_attributes(grafo_nx, w, "weight")

assert 1 == np.unique(list(nx.get_edge_attributes(grafo_nx, "weight").values())).__len__()

## Add concentrations

In [23]:


def get_largest_cc(G):
    
  largest_wcc = max(nx.connected_components(nx.Graph(G)), key=len)


  # Create a subgraph SG based on G
  SG = G.__class__()
  SG.add_nodes_from((n, G.nodes[n]) for n in largest_wcc)


  SG.add_edges_from((n, nbr, d)
      for n, nbrs in G.adj.items() if n in largest_wcc
      for nbr, d in nbrs.items() if nbr in largest_wcc)

  SG.graph.update(G.graph)

  assert G.nodes.__len__() >= SG.nodes.__len__()
  assert G.edges.__len__() >= SG.edges.__len__()
  assert SG.nodes.__len__() == len(largest_wcc)
  #assert not SG.is_directed() 
  assert nx.is_connected(nx.Graph(SG))

  return copy.deepcopy(SG)
to_remove = pd.read_csv('./results/dataframes/to_remove.csv').loc[:,"0"].tolist()
grafo_nx.remove_nodes_from(to_remove)





grafo_nx = get_largest_cc(grafo_nx)

nx.write_gpickle(grafo_nx, "./results/graphs/NX_recon_graph.gpickle")


grafo_nx_onlyConcen  = add_metabolite_concentration_features(grafo_nx, feature_data, feature_names)
print(f"{set(feature_names.Recon3_ID) - set(list(grafo_nx_onlyConcen.nodes)).intersection(set(feature_names.Recon3_ID))} is not in Recon2")


pyg_graph_onlyConcen = make_PYG_graph_from_grafo_nx(grafo_nx_onlyConcen)


{'sucaceto_c'} is not in Recon2


In [3]:
labels                 = torch.tensor(feature_data.label).reshape(len(feature_data.label),1)
pyg_graph_onlyConcen.y = labels
torch.save(pyg_graph_onlyConcen, "./results/graphs/PYG_graph_only_Concen.pt")

## Add fluxes

In [4]:
flux_samples  = pd.read_parquet("./results/flux/flux_samples.parquet.gzip")
grafo_nx_Concen_plus_Fluxes = add_flux_features(grafo_nx_onlyConcen, flux_samples, feature_data)
pyg_graph_Concen_plus_Fluxes = make_PYG_graph_from_grafo_nx(grafo_nx_Concen_plus_Fluxes)
pyg_graph_Concen_plus_Fluxes.y = labels
torch.save(pyg_graph_Concen_plus_Fluxes, "./results/graphs/PYG_graph_Concen_plus_Fluxes.pt")

In [5]:
nx.write_gpickle(grafo_nx_Concen_plus_Fluxes, "./results/graphs/NX_graph_Concen_plus_Fluxes.gpickle")